In [9]:
import pandas as pd
import requests
import time

# 1. Get Top 3 Coins
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {"vs_currency": "usd", "order": "market_cap_desc", "per_page": 3, "page": 1}
coins = requests.get(url, params=params).json()

# 2. Get Historical Data & Build Records
records = []
for coin in coins:
    print(f"Fetching: {coin['id']}...")
    hist_url = f"https://api.coingecko.com/api/v3/coins/{coin['id']}/market_chart"
    data = requests.get(hist_url, params={"vs_currency": "usd", "days": 365}).json()
    
    for p in data.get("prices", []):
        records.append({
            "coin": coin['id'],
            "date": pd.to_datetime(p[0], unit="ms").date(),
            "price": p[1]
        })
    time.sleep(10) # Respect rate limits

# 3. Create DataFrame
top3_df = pd.DataFrame(records)
print(top3_df.head())

Fetching: bitcoin...
Fetching: ethereum...
Fetching: tether...
      coin        date         price
0  bitcoin  2025-02-12  95739.977371
1  bitcoin  2025-02-13  97836.188561
2  bitcoin  2025-02-14  96561.663999
3  bitcoin  2025-02-15  97488.481485
4  bitcoin  2025-02-16  97569.951694


In [10]:
top3_df

,coin,date,price
0,bitcoin,2025-02-12,95739.977371
1,bitcoin,2025-02-13,97836.188561
2,bitcoin,2025-02-14,96561.663999
3,bitcoin,2025-02-15,97488.481485
4,bitcoin,2025-02-16,97569.951694
...,...,...,...
1093,tether,2026-02-08,0.999431
1094,tether,2026-02-09,0.999499
1095,tether,2026-02-10,0.999576
1096,tether,2026-02-11,0.999498


In [11]:
top3_df=top3_df.dropna()
top3_df

,coin,date,price
0,bitcoin,2025-02-12,95739.977371
1,bitcoin,2025-02-13,97836.188561
2,bitcoin,2025-02-14,96561.663999
3,bitcoin,2025-02-15,97488.481485
4,bitcoin,2025-02-16,97569.951694
...,...,...,...
1093,tether,2026-02-08,0.999431
1094,tether,2026-02-09,0.999499
1095,tether,2026-02-10,0.999576
1096,tether,2026-02-11,0.999498


In [15]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="jujo",
  auth_plugin='mysql_native_password'
)

mycursor = mydb.cursor()

In [2]:
mycursor.execute("create database crypto_analysis")

In [16]:
mycursor.execute("use crypto_analysis")

In [9]:
mycursor.execute("""
    CREATE TABLE historical_Prices (
        coin_id VARCHAR(50),
        `date` DATE,
        price_usd DECIMAL(18, 6),
        PRIMARY KEY (coin_id, `date`), 
        CONSTRAINT fk_coin 
            FOREIGN KEY (coin_id) 
            REFERENCES Cryptocurrencies(id)
    )
""")

In [23]:
unique_ids = top3['coin'].unique()
print(unique_ids)

['bitcoin' 'ethereum' 'tether']


In [25]:
mycursor.execute("ALTER TABLE historical_prices DROP FOREIGN KEY fk_coin")

In [26]:
prices = """
INSERT INTO historical_Prices (coin_id, `date`, price_usd) 
VALUES (%s, %s, %s)
ON DUPLICATE KEY UPDATE price_usd = VALUES(price_usd)
"""

data_to_insert = [tuple(x) for x in df[['coin', 'date', 'price']].values]

mycursor.executemany(prices, data_to_insert)
mydb.commit()

print(f"Successfully inserted {mycursor.rowcount} rows!")

Successfully inserted 1101 rows!


In [ ]:
#historical_Prices #Query 1 

In [27]:
query_1 = "SELECT coin_id,`date`, price_usd FROM historical_Prices where coin_id = 'bitcoin' ORDER BY date DESC LIMIT 1"

mycursor.execute(query_1)

highest_daily_price = mycursor.fetchall()

print("highest_daily_price:")
for row in highest_daily_price:
    print(f" coin_id: {row[0]} | `date`: {row[1]} | price_usd: {row[2]}")

highest_daily_price:
 coin_id: bitcoin | `date`: 2026-02-11 | price_usd: 67172.842926


In [ ]:
#historical_Prices #Query 2 average value for each day

In [4]:

query_2 = """
SELECT coin_id, `date`, AVG(price_usd) as average_daily_price 
FROM historical_Prices 
WHERE coin_id = 'ethereum' AND `date` >= DATE_SUB(CURDATE(), INTERVAL 1 YEAR)
GROUP BY `date`, coin_id
ORDER BY `date` DESC
"""

mycursor.execute(query_2)


avg_daily_price = mycursor.fetchall()

print("Average Daily Price for Ethereum:")
for row in avg_daily_price:

     print(f" coin_id: {row[0]} | Date: {row[1]} | Avg Price: {row[2]:,.2f}")

Average Daily Price for Ethereum:
 coin_id: ethereum | Date: 2026-02-11 | Avg Price: 1,959.48
 coin_id: ethereum | Date: 2026-02-10 | Avg Price: 2,104.46
 coin_id: ethereum | Date: 2026-02-09 | Avg Price: 2,095.13
 coin_id: ethereum | Date: 2026-02-08 | Avg Price: 2,091.04
 coin_id: ethereum | Date: 2026-02-07 | Avg Price: 2,060.73
 coin_id: ethereum | Date: 2026-02-06 | Avg Price: 1,820.57
 coin_id: ethereum | Date: 2026-02-05 | Avg Price: 2,152.09
 coin_id: ethereum | Date: 2026-02-04 | Avg Price: 2,226.99
 coin_id: ethereum | Date: 2026-02-03 | Avg Price: 2,344.51
 coin_id: ethereum | Date: 2026-02-02 | Avg Price: 2,269.33
 coin_id: ethereum | Date: 2026-02-01 | Avg Price: 2,443.93
 coin_id: ethereum | Date: 2026-01-31 | Avg Price: 2,702.41
 coin_id: ethereum | Date: 2026-01-30 | Avg Price: 2,818.82
 coin_id: ethereum | Date: 2026-01-29 | Avg Price: 3,006.81
 coin_id: ethereum | Date: 2026-01-28 | Avg Price: 3,021.09
 coin_id: ethereum | Date: 2026-01-27 | Avg Price: 2,927.84
 coin_

In [ ]:
#historical_Prices #Query 3

In [29]:
target_coin = 'bitcoin' 

query = """
SELECT coin_id, AVG(price_usd), MIN(price_usd), MAX(price_usd)
FROM historical_Prices
WHERE coin_id = %s
  AND `date` >= DATE_SUB(CURDATE(), INTERVAL 1 YEAR)
GROUP BY coin_id
"""

mycursor.execute(query, (target_coin,))
result = mycursor.fetchone()

if result:
    print(f"--- {result[0].upper()} YEARLY STATS ---")
    print(f"Average Price: ${result[1]:,.2f}")
    print(f"Yearly High:   ${result[3]:,.2f}")
    print(f"Yearly Low:    ${result[2]:,.2f}")
else:
    print(f"No data found for {target_coin} in the last year.")

--- BITCOIN YEARLY STATS ---
Average Price: $100,103.50
Yearly High:   $124,773.51
Yearly Low:    $62,853.69


In [ ]:
#historical_Prices #Query 4

In [30]:
query = """
SELECT coin_id, AVG(price_usd) AS avg_price
FROM historical_Prices
WHERE `date` >= DATE_SUB(CURDATE(), INTERVAL 1 YEAR)
GROUP BY coin_id
ORDER BY avg_price DESC
LIMIT 1
"""

mycursor.execute(query)
highest_average_price = mycursor.fetchone()

if result:
    print(f"The coin with the highest average price is: {result[0]}")
    print(f"Average Price: ${result[1]:,.2f}")
else:
    print("No data found for the specified period.")

The coin with the highest average price is: bitcoin
Average Price: $100,103.50


In [ ]:
#historical_Prices #Query 5

In [31]:
# 1. Fetch data for both September 2024 and September 2025
query_5 = """
SELECT `date`, price_usd 
FROM historical_Prices 
WHERE coin_id = 'bitcoin' 
  AND (`date` LIKE '2024-09-%' OR `date` LIKE '2025-09-%')
ORDER BY `date` ASC
"""

mycursor.execute(query_5)
data = mycursor.fetchall()

if len(data) >= 2:
    # Use the first available date in Sep 2024 and the last in Sep 2025
    price_start = data[0][1]
    price_end = data[-1][1]
    
    # Calculate % Change
    percent_change = ((price_end - price_start) / price_start) * 100
    
    print(f"Bitcoin Price (Start of Sep 2024): ${price_start:,.2f}")
    print(f"Bitcoin Price (End of Sep 2025):   ${price_end:,.2f}")
    print(f"Total Percentage Change:           {percent_change:.2f}%")
else:
    print("Insufficient data found in the database for those months.")

Bitcoin Price (Start of Sep 2024): $108,253.36
Bitcoin Price (End of Sep 2025):   $114,309.15
Total Percentage Change:           5.59%
